# Practical 8: Waste Input-Output

Objectives
- Understand the structure of a WIOT
- Build a WIO model in Python
- Analyze data from a WIO model

The methods used in this practical sessions come from:
Nakamura, S. and Kondo, Y. (2002), Input-Output Analysis of Waste Management. Journal of Industrial Ecology, 6: 39-63. https://doi.org/10.1162/108819802320971632

In [1]:
# Import packages

import pandas as pd
import numpy as np

## Exercise 1: Conceptual understanding of waste input output tables and their treatment of economic and physical flows. 

<table>
<tr><td>

|   |  E  | WT |
|---|-----|----|
| E | 52  | 22 |
| WT| 349 | 44 |

</td><td>

|  y  |
|-----|
| 100 |
| 50  |

</td></tr> </table>

<em>Table 1: WIOT of a country</em>



### Question 1.1: What are the expected units in the following parts of the simplified WIO system shown above? 

- E-WT quadrant (i.e., with value 22)
- WT-WT quadrant (i.e., with value 44)
- and in Y (i.e., with values 100 and 50)

### Question 1.2: Estimate the A-matrix. What do the coefficient represent in each quadrant? 



In [2]:
Z = np.array([[52,22],[349,44]])
y = np.array([100,50])
x = Z.sum(axis=1)+ y
inv_diag_x = np.linalg.inv(np.diag(x)) 
A = Z @ inv_diag_x
A,x

(array([[0.29885057, 0.0496614 ],
        [2.00574713, 0.0993228 ]]),
 array([174, 443]))

## Exercise 2: Construct a Waste Input-Output model in Python and analyse the waste streams linked to economic activity

Use the data provided and the following figures to help you

- _Data adapted from source: https://www.f.waseda.jp/nakashin/WIO/wio2000sec103_0.06b_eng.xls_
- _For additional info: http://www.f.waseda.jp/nakashin/WIO.html_

<table>
<tr  style='border-collapse: collapse'>
<td>

|      **Z**      |   Economic sector    |    Waste treatment     |
|-----------------|----------------------|------------------------|
| Economic sector | **Z**<sub>E,E</sub>  |  **Z**<sub>E,WT</sub>  |
| Waste treatment | **Z**<sub>WT,E</sub> |  **Z**<sub>WT,WT</sub> |
-------------------------------------------------------------------
|      **r**'     |                      |                        |
|-----------------|----------------------|------------------------|
|   Emissions     | Econ. Sec. emissions | Waste Treat. emissions |

</td>
<td>

|  **Y**             |
|--------------------|
|  **Y**<sub>E</sub> |
| **Y**<sub>WT</sub> |

----------------------
| **h**              |
|--------------------|
|      Emissions     |

</td>
</tr> 
</table>
<em>Table 2: Basic structure of a WIOT</em>



| ![Waste Input-Output Accounting structure](figures/wio_1.gif) | 
|:--:| 
| *Waste Input-Output Accounting structure (Nakamura and Kondo, 2009)* |

#### 2.1 Import data

In [3]:
data = 'data/Japan_WIO2000sec103.xlsx'
# Input-output table, including intermediate and final demand matrix
ZYeconomy = pd.read_excel(data, sheet_name = 'ZYeconomy', index_col = 0)
# Waste account, including intermediate and final demand matrix
ZYwaste = pd.read_excel(data, sheet_name = 'ZYwaste', index_col = 0)
# VA and satellite account
F = pd.read_excel(data, sheet_name = 'F', index_col = 0)
# Waste allocation matrix
Q = pd.read_excel(data, sheet_name = 'Allocation', index_col = 0)
# units
unit = pd.read_excel(data, sheet_name = 'Units', index_col = False) 

#### 2.2 identify the number of sectors and sector headers

In [4]:
n_e = ZYeconomy.shape[0] # 103 number of economic sectors
n_wt = Q.shape[0] # 13 number of waste treatment sectors
n_y = ZYeconomy.shape[1] - n_e - n_wt # 11 number of final demand categories
n_w = Q.shape[1] # 79 number of waste flows

### 2.3 Separate the following variables:

Look at the provided tables and separate the following elements: 

- **Z_e_e**: The inter-industry transactions of the economic transactions (i.e., IO economy with the 103 economic sectors)
- **Z_e_wt**: The inter-industry transactions concerning waste (i.e., the IO 13 waste treatment categories)

- **Y**: The final demand matrix (i.e., the 11 final demand categories)

- **Z_wo_e**: The waste outflows of the 103 economic sectors
- **Z_wi_e**: The waste inflows of the 103 economic sectors

- **Z_wo_wt**: The waste outflows of the 13 waste treatments 
- **Z_wi_wt**: The waste inflows of the 13 waste treatments

- **Y_wi**: The waste inflows of the 11 final demand categories
- **Y_wo**: The waste outflows of the 11 final demand categories

In [5]:
Z_e_e = ZYeconomy.iloc[:,:n_e] # IO economy with the 103 economic sectors
Z_e_wt = ZYeconomy.iloc[:, n_e:n_e + n_wt] # IO waste treatment with the 13 waste treatment categories
Y_e = ZYeconomy.iloc[:, n_e+n_wt:n_e+n_wt+n_y] # Final demand matrix

Z_wo_e = ZYwaste.iloc[:n_w, :n_e] # Waste outflow of 103 economic sectors
Z_wi_e = ZYwaste.iloc[n_w:n_w+n_w, :n_e] # Waste inflow of 103 economic sectors

Z_wo_wt = ZYwaste.iloc[:n_w, n_e:n_e+n_wt] # Waste outflow of 13 waste treatments
Z_wi_wt = ZYwaste.iloc[n_w:n_w+n_w, n_e:n_e+n_wt] # Waste inflow of 13 waste treatments

Y_wo = ZYwaste.iloc[:n_w, n_e+n_wt:n_e+n_wt+n_y] # Waste outflow of 11 final demand categories
Y_wi = ZYwaste.iloc[n_w:n_w+n_w, n_e+n_wt:n_e+n_wt+n_y] # Waste inflow of 11 final demand categories


### 2.4 Calculate net waste flows: 


**Z_wf_e**: Net waste flow of 103 economic sectors

- given by $\mathbf{Z}_{WF,E} = \mathbf{Z}_{WFO,E} - \mathbf{Z}_{WFI,E}$
- where $\mathbf{Z}_{WF,E}$ are the net waste flows for economic sector E,
- $\mathbf{Z}_{WFO,E}$ are the waste outflows for economic sector E
- and $\mathbf{Z}_{WFI,E}$ is the waste inflow for economic sector E;

**Z_wf_wt**: Net waste flow of 13 waste treatmets
- given by $\mathbf{Z}_{WF,WT} = \mathbf{Z}_{WFO,WT} - \mathbf{Z}_{WFI,WT}$
- where $\mathbf{Z}_{WF,WT}$ are the net waste flows for waste treatment sector WT, 
- $\mathbf{Z}_{WFO,WT}$ are the waste outflows for waste treatment sector WT, 
- and $\mathbf{Z}_{WFI,WT}$ is the waste inflow for waste treatment sector WT;

**Y_wf**: Net waste flow of 11 final demand categories
- given by $\mathbf{Y}_{WF} = \mathbf{Y}_{WFO} - \mathbf{Y}_{WFI}$
- where $\mathbf{Y}_{WF}$ are the net waste flows for final demand Y, 
- $\mathbf{Y}_{WFO}$ is the waste outflows for final demand Y, 
- and $\mathbf{Y}_{WFI}$ is the waste inflow for final demand Y.



In [6]:
Z_wf_e_net = Z_wo_e - Z_wi_e.values # Net waste flow of 103 economic sectors.
Z_wf_wt_net = Z_wo_wt - Z_wi_wt.values # Net waste flow of 13 waste treatmets
Y_wf_net = Y_wo - Y_wi.values # Net waste flow of 11 final demand categories

### 2.5 Covert waste flows into waste treatments:

**Z_wt_e**: Matrix of waste flows generated and/or absorbed by 103 economic sector
- given by $\mathbf{Z}_{WT,E} = \mathbf{Q} \mathbf{Z}_{WF,E}$
- where $\mathbf{Z}_{WT,E}$ is a matrix of waste flows generated and/or absorbed by economic sector E
- and $\mathbf{Q}$ is the allocation matrix;

**Z_wt_wt**: Matrix of waste flows generated and/or absorbed by 13 waste treatment sectors sector
- given by $\mathbf{Z}_{WT,WT} = \mathbf{Q} \mathbf{Z}_{WF,WT}$
- where $\mathbf{Z}_{WT,WT}$ is a matrix of waste flows generated and/or absorbed by waste treatment sector WT;

**Y_wt**: Matrix of waste flows generated and/or absorbed by 11 final demand categories
- given by $\mathbf{Y}_{WT} = \mathbf{Q} \mathbf{Y}_{WF}$
- where $\mathbf{Y}_{WT}$ is a matrix of waste flows generated by final demand $\mathbf{Y}$.

In [9]:
Z_wt_e = Q.values @ Z_wf_e_net  # Matrix of waste flows generated and/or absorbed by 103 economic sector
Z_wt_wt = Q.values @ Z_wf_wt_net  # Matrix of waste flows generated and/or absorbed by 13 waste treatment sectors sector
Y_wt = Q.values @ Y_wf_net  # Matrix of waste flows generated and/or absorbed by 11 final demand categories

# # Reapply labels

Z_wt_e.index = Q.index
Z_wt_wt.index = Q.index
Y_wt.index = Q.index

### 2.6 Group matrices in a WIOT arrangement

In [10]:
wiot_e = pd.concat([Z_e_e, Z_e_wt], axis=1)  # Concat Z_e_e and Z_e_wt block
wiot_wt = pd.concat([Z_wt_e, Z_wt_wt], axis=1) # Concat Z_wt_e and Z_wt_wt block

Z_wiot = pd.concat([wiot_e, wiot_wt], axis=0) # Integrate WIOT
Y_wiot = pd.concat([Y_e, Y_wt], axis=0) # Concat Y_e and Y_wt

In [11]:
Z_wiot.head()

,Crop cultivation,Livestock,Agricultural services,Forestry,Fisheries,Metallic ores,Non-metallic ores,Coal mining,Crude petroleum and natural gas,Foods,...,Landfill,Shredding: bulky textile,Shredding: wooden furniture,"Shredding: bikes, ovens",Shredding: small electric appliances,Shredding: TV sets,Shredding: refrigerators,Shredding: washing machines,Shredding: air conditioners,Shredding: automobiles
Crop cultivation,165201.999988,277271.999978,3609.000000,1361.999999,0.000000,0.000000,0.0,0.000000,0.0,3.491681e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Livestock,47292.999996,268280.999971,6748.999999,2535.999998,0.000000,0.000000,0.0,0.000000,0.0,1.974122e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Agricultural services,341458.999986,157408.999992,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Forestry,2325.000000,0.000000,0.000000,190231.999634,1172.000000,105.999999,89.0,308.999998,19.0,1.736700e+04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fisheries,0.000000,0.000000,0.000000,0.000000,93567.999996,0.000000,0.0,0.000000,0.0,1.262796e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
Y_wiot.head()

,Consumption expenditure outside households (column),Consumption expenditure of households,Consumption expenditure of private non-profit institutions serving households,Consumption expenditure of general government,Consumption expenditure of general government (social fixed capital depreciation),Gross domestic fixed capital formation (public),Gross domestic fixed capital formation (private),Increase in stocks,Exports,Exports (direct purchase),Balancing sector
Crop cultivation,66019.0,3001038.0,0.0,0.0,0.0,0.0,44191.0,19150.0,11809.0,83.0,562.0
Livestock,0.0,209839.0,0.0,0.0,0.0,0.0,149290.0,38327.0,824.0,11.0,40.0
Agricultural services,0.0,84720.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Forestry,4179.0,187005.0,0.0,0.0,0.0,0.0,0.0,679130.0,1547.0,1.0,61.0
Fisheries,21023.0,392104.0,0.0,0.0,0.0,0.0,0.0,37110.0,54924.0,47.0,2109.0


## Exercise 3: From a consumption-based perspective, which are the major contributors to landfill area footprint?

### 3.1 Calculate Leontief inverse

In [13]:
x = Z_wiot.sum(axis = 1).values + Y_wiot.sum(axis = 1).values


x_ = x.copy()
x_[x_!=0] = 1/x_[x_!=0]
inv_diag_x_ = np.diag(x_)


A = Z_wiot @ inv_diag_x_

I = np.identity(A.shape[0])

L = np.linalg.inv(I - A)

### 3.2 Calculate Landfill intensity

In [14]:
## Landfill intensity
f = F.iloc[8,: n_e+n_wt] @ inv_diag_x_

### 3.3 Calculate landfill footprint  per economic sector (LF)

In [15]:
LF = f @ L @ np.diag(Y_wiot.sum(1).values) # Landfill footprint calculation
LF = pd.DataFrame(LF, index=Y_wiot.index, columns=['Landfill footprint'])  # Create new dataframe with LF
LF_sort = LF.sort_values(by='Landfill footprint', ascending=False)  # Sort results
LF_sort.head(5) # Print top-5 contributors

,Landfill footprint
Landfill,7.417132e+07
Building construction,6.785285e+07
Water supply,1.493733e+07
Shredding: automobiles,1.487708e+07
Other civil engineering and construction,1.003893e+07


### Credits

First version of the exercise developed by dr. J.F.D. Rodrigues, improvements by dr. G.A. Aguilar Hernandez (2022) and F. Donati (2023)  